# Faostat world regions

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import dates
import matplotlib.dates as mdates
from osgeo import gdal, osr, ogr
import geopandas as gpd
from shapely.geometry import Point
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.stattools import adfuller
from tqdm import tqdm_notebook
from itertools import product
from typing import Union
import warnings
warnings.filterwarnings('ignore')
from platform import python_version
print("Python version: %s"%python_version())
print("NumPy version: %s"%np.__version__)
print("Pandas version: %s"%pd.__version__)
print('GDAL version: %s'%gdal.__version__)
print("GeoPandas version: %s"%gpd.__version__)

Python version: 3.11.5
NumPy version: 1.25.2
Pandas version: 2.1.1
GDAL version: 3.7.2
GeoPandas version: 0.14.0


## Maize yield

In [5]:
#path_fao_dataset = 'data/FAOSTAT_maize_millet_world_yeld.csv'
path_fao_dataset = 'data/FAOSTAT_maize_world_regions.csv'
maize_world_regions_tmp_df = pd.read_csv(path_fao_dataset)
maize_world_regions_tmp_df.shape

(310, 15)

In [6]:
maize_world_regions_tmp_df.head(3)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,2,Africa,5419,Yield,112,Maize (corn),1961,1961,100 g/ha,10444,E,Estimated value,NaN
1,QCL,Crops and livestock products,2,Africa,5419,Yield,112,Maize (corn),1962,1962,100 g/ha,11112,E,Estimated value,NaN
2,QCL,Crops and livestock products,2,Africa,5419,Yield,112,Maize (corn),1963,1963,100 g/ha,11094,E,Estimated value,NaN


## Temperature change on land

In [7]:
path_temperature_dataset = 'data/FAOSTAT_Annual_temp_change_on_land_world_regions.csv'
temperature_tmp_df = pd.read_csv(path_temperature_dataset)
temperature_tmp_df.shape

(10710, 14)

In [8]:
temperature_tmp_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Months Code,Months,Year Code,Year,Unit,Value,Flag,Flag Description
0,ET,Temperature change on land,2,Africa,7271,Temperature change,7001,January,1961,1961,°c,0.105,E,Estimated value
1,ET,Temperature change on land,2,Africa,7271,Temperature change,7001,January,1962,1962,°c,-0.332,E,Estimated value


## Fertilizers by nutrient (synthetic fertilizers)
Nitrogen, phosphate P205, and potash.

In [10]:
path_fertilizers_dataset = 'data/FAOSTAT_synthetic_fertilizers_world_regions.csv'
fertilizers_df = pd.read_csv(path_fertilizers_dataset)
fertilizers_df.shape

(915, 15)

In [11]:
fertilizers_df.tail(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
913,RFN,Fertilizers by Nutrient,9,Oceania,5157,Agricultural Use,3104,Nutrient potash K2O (total),2020,2020,t,435423.36,E,Estimated value,NaN
914,RFN,Fertilizers by Nutrient,9,Oceania,5157,Agricultural Use,3104,Nutrient potash K2O (total),2021,2021,t,481150.46,E,Estimated value,NaN


## Livestock manure

In [15]:
path_manure_dataset = 'data/FAOSTAT_Livestock_manure_world_regions.csv'
manure_df = pd.read_csv(path_manure_dataset, usecols=['Area', 'Year', 'Value'])
manure_df.shape

(305, 3)

In [16]:
manure_df.tail(2)

,Area,Year,Value
303,Oceania,2020,2.317287e+08
304,Oceania,2021,2.412670e+08


## Precipitations
Data about precipitations at regional level can be retr